In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
from google.cloud import bigquery
import os
from google.cloud import bigquery


# pd.set_option('display.max_columns', None)

os.chdir('/home/jupyter')
!pwd
# # %cd 
# !pwd
# (base) jupyter@starfire-1:~$ pip install jupyter-console
# (base) jupyter@starfire-1:~$ !jupyter console --existing
%connect_info



/home/jupyter
{
  "shell_port": 33609,
  "iopub_port": 54653,
  "stdin_port": 40423,
  "control_port": 52377,
  "hb_port": 36207,
  "ip": "127.0.0.1",
  "key": "244c80b5-0416ad359880ef783b590213",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "conda-root-py",
  "jupyter_session": "/home/jupyter/starfire/notebooks/data_1_refi_deals.ipynb"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-ddb5ba7b-61a5-41e9-8b2b-988411085db6.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [3]:

import refinitiv.dataplatform as rdp

%run ./starfire/notebooks/credentials.ipynb

session = rdp.open_platform_session(
    APP_KEY,
    rdp.GrantPassword(
        username = RDP_LOGIN,
        password = RDP_PASSWORD
    )
)

In [7]:

# Initialize a BigQuery client
client = bigquery.Client()

# Define your project, dataset, and table
project_id = "starfire-410116"
dataset_id = "derived"
table_id = "refinitiv_deals_3"


# Construct a reference to the target table
table_ref = client.dataset(dataset_id).table(table_id)

# Configure the load job to append data to the existing table
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND
)


master_df = pd.DataFrame()


from datetime import datetime, timedelta

# Initial start date
start_date = datetime.strptime('19850101', '%Y%m%d')
 # 20000101 20210330 20210109

# Define the interval (5 days) to avoid time outs
interval = timedelta(days=2)

# Initialize an empty list to store the date ranges
date_ranges = []

# Loop to create date intervals
for _ in range(10000):  # Adjust the range to get the desired number of intervals
    end_date = start_date + interval
    date_ranges.append((start_date.strftime('%Y%m%d'), end_date.strftime('%Y%m%d')))
    start_date = end_date + timedelta(days=1)

j = 1
N = 21
# Print the date ranges
for start, end in date_ranges:
    success = False
    while not success:
        try:
            print(f"Start Date: {start}, End Date: {end}")

            print("Downloading data from refinitiv...")

            mna_screen = f'SCREEN(U(IN(DEALS)/*UNV:DEALSMNA*/),\
                        BETWEEN(TR.MnAAnnDate,{start},{end})/*dt:Date*/, \
                        CURN=USD)'


            # input ipo screen results as instrument list and pull ipo attributes via fields from Eikon Data API. 
            mnas = rdp.get_data(mna_screen,['TR.MNASDCDealNumber',
                                            'TR.MnAAnnDate', 'TR.MnARankValueIncNetDebt(Curn=USD,Scale=6)',
                                            'TR.MnATarget', 'TR.MnATargetPermId',
                                            'TR.MnATargetPriTickerSym',
                                            'TR.MnATargetNation',
                                            'TR.MnATargetMacroInd',
                                            'TR.MnATargetMidInd',
                                            'TR.MnAAcquiror',
                                            'TR.MnAAcquirorPermId',
                                            'TR.MnAAcquirorMacroInd',
                                            'TR.MnAAcquirorMidInd',
                                            'TR.MnAAcquirorNation',
                                            'TR.MNADealId','TR.MnASynopsis',
                                            'TR.MnAAcquirorPriHiTech',
                                            'TR.MnAAcquirorPriTickerSym',
                                            'TR.MnAAcquirorExchTicker(Concat="|")',
                                            'TR.MnAAcquirorUltParentPermID',
                                            'TR.MnAHiTechGroup(Concat="|")',
                                            'TR.MnaAcquirorDate1DayPriorToAnn',
                                            'TR.MnaAcquirorStockPrice1DayPriorToAnnInAcqCurr(Curn=USD)',
                                            'TR.MnAAcquirorStockPriceOnAnn(Curn=USD)',
                                            'TR.MnAAcquirorStockPrice1DayPostAnn(Curn=USD)',
                                            'TR.MnASnP5001DayPreAnn',
                                            'TR.MnASnP500AtAnnouncement',
                                            'TR.MnASnP5001DayPostAnn',
                                            'TR.MnATargetSDCCusip',
                                            'TR.MnAAcquirorSDCCusip',
                                            'TR.MnAAcquirorUltParentSDCCusip',
                                            'TR.MnAValueOfCash(Curn=USD, Scale=6)',
                                            'TR.MnAValueOfEarnout(Curn=USD, Scale=6)',
                                            'TR.MnADealValue(Curn=USD, Scale=6)',
                                            'TR.MnAAcquirorHiTech(Concat="|")',
                                            'TR.MnAAcquirorUltParentHiTech(Concat="|")',
                                            'TR.MnATargetPriHiTech',
                                            'TR.MnAAcquirorMajorInd',
                                            'TR.MnAAcquirorMajorIndGrp',
                                            'TR.MnAAcquirorPriVEIC',
                                            'TR.MnAMasterDealType',
                                            'TR.MnADealType',
                                            'TR.MnAType(Concat="|")',
                                            'TR.MnAAcquirorVeic(Concat="|")',
                                            'TR.MnAPurposeDescription',
                                            'TR.MnAIsCrossBorder',
                                            'TR.MnAAcquirorIsVentureFirm',
                                            'TR.MnATargetPriVEIC',
                                            'TR.MnATargetVeic(Concat="|")',
                                            'TR.MnAAcquirorBusDesc',
                                            'TR.MnATargetBusDesc',
                                            'TR.MnATargetExchTicker(Concat="|")',
                                            'TR.MnATargetUltParentSDCCusip',
                                            'TR.MnATargetUltParentPriTickerSym',
                                            'TR.MnATargetUltParentPermID',
                                            'TR.MnAAcquirorUltParentPriTickerSym',
                                            'TR.MnAAcquirorUltParentExchTicker(Concat="|")',
                                            'TR.MnATargetUltParentExchTicker(Concat="|")',
                                            'TR.MnATargetMajorInd',
                                            'TR.MnATargetMajorIndGrp',
                                            'TR.MnATargetHiTech(Concat="|")',
                                            'TR.MnAAcquirorUltParentPriHiTech',
                                            'TR.MnAAcquirorHiTechGroup(Concat="|")',
                                            'TR.MnATargetHiTechGroup(Concat="|")',
                                            'TR.MnAWithdrawnDate',
                                            'TR.MnADateUnconditional',
                                            'TR.MnAOrigAnnDate',
                                            'TR.MnAPercentCash',
                                            'TR.MnADealCurrency',
                                            'TR.MnAEarnoutDesc']) 

            print(mnas.shape)
            mnas.columns = [col[3:].lower() for col in mnas.columns]

            master_df = pd.concat([master_df, mnas], ignore_index=True)
            print(f'Master shape: {master_df.shape}')

            j += 1
            success = True 

                
            # Process some code every N iterations
            if j % N == 0:
                print(f"Appending chunk to BQ")
                # Load the DataFrame into BigQuery
                load_job = client.load_table_from_dataframe(master_df, table_ref, job_config=job_config)
                # Wait for the load job to complete
                load_job.result()
                print("Data appended successfully")
                
                # Reinitialize master_df to not store duplicate info next time this runs
                master_df = pd.DataFrame()
                j = 1
        
            
        except Exception as e:
            print(f"An error occurred: {e}. Retrying for the range {start} to {end}...")
            
print("Uploading data to BQ...")


            


Start Date: 19850101, End Date: 19850103
(26, 71)
Master shape: (26, 71)
Start Date: 19850104, End Date: 19850106


ERROR!!! An error occurred while requesting URL('https://api.refinitiv.com/data/datagrid/beta1/').
     ReadTimeout('')


An error occurred: . Retrying for the range 19850104 to 19850106...
Start Date: 19850104, End Date: 19850106
(15, 71)
Master shape: (41, 71)
Start Date: 19850107, End Date: 19850109


/var/tmp/ipykernel_10925/3801902500.py:128: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  master_df = pd.concat([master_df, mnas], ignore_index=True)


(57, 71)
Master shape: (98, 71)
Start Date: 19850110, End Date: 19850112


...

ERROR!!! An error occurred while requesting URL('https://api.refinitiv.com/data/datagrid/beta1/').
     ReadTimeout('')


An error occurred: . Retrying for the range 20000316 to 20000318...
Start Date: 20000316, End Date: 20000318
(351, 71)
Master shape: (4655, 71)
Start Date: 20000319, End Date: 20000321
(331, 71)
Master shape: (4986, 71)
Start Date: 20000322, End Date: 20000324
(545, 71)
Master shape: (5531, 71)
Start Date: 20000325, End Date: 20000327


ERROR!!! An error occurred while requesting URL('https://api.refinitiv.com/data/datagrid/beta1/').
     ReadTimeout('')


An error occurred: . Retrying for the range 20000325 to 20000327...
Start Date: 20000325, End Date: 20000327


KeyboardInterrupt: 